# BEVLocalizer — Minimal End-to-End Demo

This notebook demonstrates the BEVLocalizer orchestrator with a synthetic database:
- Generate a BEV image from random LiDAR-like points
- Extract descriptors via REIN
- Build a small retrieval index (PCA+FAISS) around the query descriptor
- Run `BEVLocalizer.localize` and (optionally) estimate relative pose


In [1]:
import numpy as np
import torch

from bevplace import REIN
from bevplace.core.types import BEVParams
from bevplace.pipeline.localizer import BEVLocalizer
from bevplace.preprocess.bev import bev_density_image_torch
from bevplace.retrieval import BEVIndex

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [2]:
# 1) Make BEV from random point cloud
params = BEVParams(D=40.0, g=0.4)
pts = torch.randn(200000, 3, device=DEVICE) * 30.0
bev = bev_density_image_torch(pts, params)
bev.shape, bev.min().item(), bev.max().item()

(torch.Size([1, 200, 200]), 0.0, 1.0)

In [3]:
# 2) Extract descriptor with REIN
model = REIN().to(DEVICE).eval()
with torch.no_grad():
    _, rem_map_q, q_desc = model(bev.unsqueeze(0))
q_desc.shape

torch.Size([1, 8192])

In [4]:
# 3) Build tiny synthetic DB around the query descriptor
rng = np.random.default_rng(0)
base = q_desc.detach().cpu().numpy()
DB = np.vstack([base + rng.normal(scale=0.1, size=base.shape) for _ in range(8)])

index = BEVIndex(pca_dim=16)
index.fit_pca(DB)
index.add(DB, poses=None)
"DB size:", index._index.ntotal

('DB size:', 8)

In [5]:
# 4) Define reference provider (returns same BEV and REM map for the matched id)
def reference_provider(_id: int):
    return bev, rem_map_q


localizer = BEVLocalizer(
    model=model, index=index, bev_params=params, device=DEVICE, reference_provider=reference_provider
)
res = localizer.localize(pts)
res.matched_id, res.topk, res.inliers_ratio, res.num_matches, res.timings

(1,
 (1,),
 1.0,
 2000,
 {'bev_ms': 0.8857157081365585,
  'rein_ms': 19.770991057157516,
  'retrieval_ms': 0.11469656601548195,
  'pose_ms': 397.1246499568224})

In [6]:
res

LocalizationResult(pose_global=None, pose_relative=array([[1.0000000e+00, 4.0887365e-09, 0.0000000e+00],
       [4.0887365e-09, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]], dtype=float32), matched_id=1, topk=(1,), distances=(70.5427017211914,), inliers_ratio=1.0, num_matches=2000, timings={'bev_ms': 0.8857157081365585, 'rein_ms': 19.770991057157516, 'retrieval_ms': 0.11469656601548195, 'pose_ms': 397.1246499568224}, descriptor_q=array([[ 0.00115945, -0.00049789, -0.00434839, ..., -0.00243711,
         0.00189914, -0.00776396]], shape=(1, 8192), dtype=float32))